In [1]:
from scipy.fft import fft, ifft, fftfreq, fftshift
import numpy as np
import matplotlib.pyplot as plt

In [20]:
#чтение спектра из файла

#out_wl -индикатор выводы массива длин волн

def read_spectr(file_name, out_wl = 1):
    lamda, spectrum = [], []
    with open(file_name, 'r') as f:
        for line in f.readlines()[2:]:
            col1, col2 = line.split()
            lamda.append(float(col1)/1000), spectrum.append(10**((float(col2)+120)/10))
    f.close()
    if out_wl == 1:
        return np.array(lamda), np.array(spectrum)
    else:
        return np.array(spectrum)

In [11]:
#сглаживающая функция

#box_pts - количесвто элементов в сглаживании 

def smooth(y, box_pts):
    box = np.ones(box_pts)/box_pts
    y_smooth = np.convolve(y, box, mode='same')
    return y_smooth

In [12]:
#интерполлирующие функции для увеличения разрешения OSA

def f_line(a, b, res):
    return [np.round(a + (b-a)*i/res, 5) for i in range(res)] #res - количество точек между исходными, включая границы

#dl - количесвто новых точек на отрезке между двумя первоначальными 
def interpolate(l_in, dl):
    l_out = []
    for i in range(len(l_in)-1):
        l_out.extend(f_line(l_in[i], l_in[i+1], dl))
    l_out.append(l_in[-1])
    return np.array(l_out)

In [49]:
#Фурье преобразование

#spectr_initial, spectr_new - начальный спектр и с интреференцией
#wo_noise1, wo_noise2 - диапазон элементов массива длин волн wavelength_mas где нет сильных шумов
#out - интдикатор вывода массивов фурье элементов

def foutier(wavelength_mas, spectr_initial, spectr_new, wo_noise1, wo_noise2, out = 0):
    
    N = len(wavelength_mas)-1 
    k = np.linspace(1/wavelength_mas[-1], 1/wavelength_mas[0], N, endpoint=False)

    ##############Выборка шумов#################
    noise_a, noise_b  = wo_noise1, wo_noise2

    Nd = noise_b - noise_a - 1
    xd = k[noise_a:noise_b]
    Td = (k[noise_b] - k[noise_a]) / Nd 
    xdf = fftfreq(Nd, Td)[:Nd//2]

    #функция интерференции
    y = (spectr_new/spectr_initial)[noise_a:noise_b]

    #Fourier function
    yf = ifft(y)
    yf_abs = 2.0/Nd * np.abs(yf[0:Nd//2])

    ###############################################

    
    plt.plot(xdf, yf_abs, '-r')
    plt.title('Fourier Domain')
    plt.xlabel('$\Delta$z, um')
    plt.ylabel('Amplitude, a.u')
    plt.grid()
    plt.show()
    if out == 1:
        return xdf, yf_abs

# ПРИМЕР

In [53]:
wl_init, sp_init = read_spectr('740mA_empty_2023_02_21_15_51_38.dat')
sp_init_sm = smooth(sp_init, 5) #cглаженный
sp_init_sm_itp = interpolate(sp_init_sm, 5) #сглаженный+больше разрешение

wl = interpolate(wl_init, 5) #массив с длинами волн

sp_new = read_spectr('740mA_glass_2023_02_21_15_50_53.dat', out_wl = 0)   
spec_new_itp = interpolate(sp_new, 5) #больше разрешение

noise_a, noise_b = 500, 4000 #бубны с шумами

plt.plot(wl[noise_a:noise_b], (spec_new_itp/sp_init_sm_itp)[noise_a:noise_b], 'm')
plt.grid()
plt.show()

x , y = foutier(wl, sp_init_sm_itp, spec_new_itp, noise_a, noise_b, out = 1)
%matplotlib qt
#%matplotlib inline # - если не хочется в отдельном окне график
plt.plot(x, y)
plt.grid()
plt.show()